# HW1 布尔查询之BSBI与索引压缩

本次作业使用斯坦福大学[CS 276 / LING 286: Information Retrieval and Web Search](https://web.stanford.edu/class/cs276/)课程的代码框架来实现。具体来说主要包含的内容有：
1. [索引构建 (40%)](#索引构建与检索-(40%)) 使用BSBI方法模拟在内存不足的情况下的索引构建方式，并应用于布尔查询
2. [索引压缩 (30%)](#索引压缩-(30%)) 使用可变长编码对构建的索引进行压缩
3. [布尔检索 (10%)](#布尔联合检索-(10%)) 对空格分隔的单词查询进行联合（与）布尔检索
3. [实验报告 (10%)](#Report-(25%)) 描述你的代码并回答一些问题
4. [额外的编码方式 (10%)](#额外的编码方式-(10%)) 鼓励使用额外的编码方式对索引进行压缩 (例如, gamma-encoding)

In [805]:
# You can add additional imports here
import sys
import pickle as pkl
import array
import os
import timeit
import contextlib

# 数据集

实验使用的文本数据是stanford.edu域下的网页内容，可从http://web.stanford.edu/class/cs276/pa/pa1-data.zip 下载。以下代码将大约170MB的文本数据下载到当前目录下，

In [806]:
"""import urllib.request
import zipfile

data_url = 'http://web.stanford.edu/class/cs276/pa/pa1-data.zip'
data_dir = 'pa1-data'
urllib.request.urlretrieve(data_url, data_dir+'.zip')
zip_ref = zipfile.ZipFile(data_dir+'.zip', 'r')
zip_ref.extractall()
zip_ref.close()"""

"import urllib.request\nimport zipfile\n\ndata_url = 'http://web.stanford.edu/class/cs276/pa/pa1-data.zip'\ndata_dir = 'pa1-data'\nurllib.request.urlretrieve(data_url, data_dir+'.zip')\nzip_ref = zipfile.ZipFile(data_dir+'.zip', 'r')\nzip_ref.extractall()\nzip_ref.close()"

之后构建的索引会被存储到`output_dir`，`tmp`会存储测试数据（toy-data）所生成的一些临时文件

In [807]:
try: 
    os.mkdir('output_dir')
except FileExistsError:
    pass
try: 
    os.mkdir('tmp')
except FileExistsError:
    pass
try: 
    os.mkdir('toy_output_dir')
except FileExistsError:
    pass

在数据目录下有10个子目录（命名0-9）

In [808]:
sorted(os.listdir('pa1-data'))

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

每一个子目录下的文件都包含一个独立网页的内容。可以认为在同一子目录下没有同名文件，即每个文件的绝对路径不会相同。

In [809]:
sorted(os.listdir('pa1-data/0'))[:10]

['3dradiology.stanford.edu_',
 '3dradiology.stanford.edu_patient_care_Case%2520studies_AVM.html',
 '3dradiology.stanford.edu_patient_care_case_studies.html',
 '5-sure.stanford.edu_',
 '50years.stanford.edu_',
 'a3cservices.stanford.edu_awards_nominate_',
 'a3cservices.stanford.edu_facilities_',
 'a3cservices.stanford.edu_lead_',
 'aa.stanford.edu_',
 'aa.stanford.edu_about_aviation.php']

所有的网页内容已经经过处理，仅包含由空格分隔开的单词，不再需要进行额外的标准化工作。

In [810]:
with open('pa1-data/0/3dradiology.stanford.edu_', 'r') as f:
    print(f.read())

3d radiology lab stanford university school of medicine stanford school of medicine 3d and quantitative imaging in the department of radiology search this site only stanford medical sites ways to give find a person alumni lane library ways to give find a person about us mission to develop and apply innovative techniques for efficient quantitative analysis and display of medical imaging data through interdisciplinary collaboration goals education to train physicians and technologists locally and worldwide in the latest developments in 3d and quantitative imaging research to develop new approaches to the exploration analysis and quantitative assesment of diagnostic images that result in a new and or more cost effective diagnostic approaches and b new techniques for the design and planning and monitoring of therapy patient care to deliver valid clinically relevant visualization and analysis of medical imaging data to the stanford community locations richard m lucas magnetic resonance imag

作业目录下有一个小的数据集文件夹`toy-data`。在使用完整网页数据集之前，我们会用它来测试我们的代码是否正确。

In [811]:
toy_dir = 'toy-data'

# 索引构建与检索 (40%)

作业的第一部分是使用**blocked sort-based indexing (BSBI)** 算法来构建倒排索引并实现布尔检索。关于BSBI算法可以参考老师课件或者斯坦福教材[Section 4.2](http://nlp.stanford.edu/IR-book/pdf/04const.pdf)。以下摘自教材内容

> To construct an index, we first make a pass through the collection assembling all term-docID pairs. We then sort the pairs with the term as the dominant key and docID as the secondary key. Finally, we organize the docIDs for each term into a postings list and compute statistics like term and document frequency. For small collections, all this can be done in memory. 

对于无法在内存一次性处理的较大数据集，将会使用到二级存储（如：磁盘）。

## IdMap

再次引用教材 Section 4.2:

> To make index construction more efficient, we represent terms as termIDs (instead of strings), where each termID is a unique serial number. We can build the mapping from terms to termIDs on the fly while we are processing the collection. Similarly, we also represent documents as docIDs (instead of strings).

我们首先定义一个辅助类`IdMap`，用于将字符串和数字ID进行相互映射，以满足我们在term和termID、doc和docID间转换的需求。

实现以下代码中的`_get_str` 和 `_get_id`函数，IdMap类的唯一接口是`__getitem__`，它是一个特殊函数，重写了下标运算`[]`,根据下标运算键的类型得到正确的映射值（如果不存在需要添加）。（特殊函数可参考[官方文档](https://docs.python.org/3.7/reference/datamodel.html#special-method-names)）
<br>
<br>
我们会用到字典来将字符串转换为数字，用列表来将数字转换为字符串。

In [812]:
class IdMap:
    """Helper class to store a mapping from strings to ids."""
    def __init__(self):
        self.str_to_id = {}
        self.id_to_str = []
        self.cnt = 0
        
    def __len__(self):
        """Return number of terms stored in the IdMap"""
        return len(self.id_to_str)
        
    def _get_str(self, i):
        """Returns the string corresponding to a given id (`i`)."""
        ### Begin your code
        for j in range(len(self.id_to_str)):
            ###这里的条件表示这里查找的id出现了，所以不会报错，且直接返回str
            if self.id_to_str[j] == i:
                for key, value in self.str_to_id.items():
                    if value == i:
                        return key
        "print(self.id_to_str)"
        raise IndexError
        ### End your code
        
    def _get_id(self, s):
        """Returns the id corresponding to a string (`s`). 
        If `s` is not in the IdMap yet, then assigns a new id and returns the new id.
        """
        ### Begin your code
        ###当不存在时，进行一个赋值
        if self.str_to_id.get(s) is None:
            self.str_to_id[s] = self.cnt
            self.id_to_str.append(self.cnt)
            self.cnt = self.cnt + 1
        "print(self.str_to_id)"
        return self.str_to_id[s]
        ### End your code
            
    def __getitem__(self, key):
        """If `key` is a integer, use _get_str; 
           If `key` is a string, use _get_id;"""
        if type(key) is int:
            return self._get_str(key)
        elif type(key) is str:
            return self._get_id(key)
        else:
            raise TypeError

确保代码能通过以下简单测试样例

In [813]:
testIdMap = IdMap()
assert testIdMap['a'] == 0, "Unable to add a new string to the IdMap"
assert testIdMap['bcd'] == 1, "Unable to add a new string to the IdMap"
assert testIdMap['a'] == 0, "Unable to retrieve the id of an existing string"
assert testIdMap[1] == 'bcd', "Unable to retrive the string corresponding to a\
                                given id"
try:
    testIdMap[2]
except IndexError as e:
    assert True, "Doesn't throw an IndexError for out of range numeric ids"
assert len(testIdMap) == 2

之后会需要你自己来写测试样例来确保你的程序正常运行

## 将倒排列表编码成字节数组

为了高效地从磁盘读写倒排列表（文档ID），我们将其存储为字节数组的形式。代码提供了`UncompressedPostings`类来用静态函数实现对倒排列表的编码和解码。在之后的任务中你需要使用该接口实现索引压缩版本（可变长编码）。

参考:
1. https://docs.python.org/3/library/array.html
2. https://pymotw.com/3/array/#module-array

In [814]:
class UncompressedPostings:
    
    @staticmethod
    def encode(postings_list):
        """Encodes postings_list into a stream of bytes
        
        Parameters
        ----------
        postings_list: List[int]
            List of docIDs (postings)
            
        Returns
        -------
        bytes
            bytearray representing integers in the postings_list
        """
        return array.array('L', postings_list).tobytes()
        
    @staticmethod
    def decode(encoded_postings_list):
        """Decodes postings_list from a stream of bytes
        
        Parameters
        ----------
        encoded_postings_list: bytes
            bytearray representing encoded postings list as output by encode 
            function
            
        Returns
        -------
        List[int]
            Decoded list of docIDs from encoded_postings_list
        """
        
        decoded_postings_list = array.array('L')
        decoded_postings_list.frombytes(encoded_postings_list)
        return decoded_postings_list.tolist()

运行以下代码查看其工作方式

In [815]:
x = UncompressedPostings.encode([1,2,3])
print(x)
print(UncompressedPostings.decode(x))

b'\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00'
[1, 2, 3]


## 磁盘上的倒排索引

> With main memory insufficient, we need to use an external sorting algorithm, that is, one that uses disk. For acceptable speed, the central requirement of such an algorithm is that it minimize the number of random disk seeks during sorting - sequential disk reads are far faster than seeks. 

在这一部分我们提供了一个基类`InvertedIndex`，之后会在此基础上构建它的子类`InvertedIndexWriter`, `InvertedIndexIterator` 和 `InvertedIndexMapper`。在Python中我们常用`cPickle`进行序列化，但是它并不支持部分读和部分写，无法满足BSBI算法的需要，所以我们需要定义自己的存储方式。

In [816]:
class InvertedIndex:
    """A class that implements efficient reads and writes of an inverted index 
    to disk
    
    Attributes
    ----------
    postings_dict: Dictionary mapping: termID->(start_position_in_index_file, 
                                                number_of_postings_in_list,
                                               length_in_bytes_of_postings_list)
        This is a dictionary that maps from termIDs to a 3-tuple of metadata 
        that is helpful in reading and writing the postings in the index file 
        to/from disk. This mapping is supposed to be kept in memory. 
        start_position_in_index_file is the position (in bytes) of the postings 
        list in the index file
        number_of_postings_in_list is the number of postings (docIDs) in the 
        postings list
        length_in_bytes_of_postings_list is the length of the byte 
        encoding of the postings list
    
    terms: List[int]
        A list of termIDs to remember the order in which terms and their 
        postings lists were added to index. 
        
        After Python 3.7 we technically no longer need it because a Python dict 
        is an OrderedDict, but since it is a relatively new feature, we still
        maintain backward compatibility with a list to keep track of order of 
        insertion.  
    """
    def __init__(self, index_name, postings_encoding=None, directory=''):
        """
        Parameters
        ----------
        index_name (str): Name used to store files related to the index
        postings_encoding: A class implementing static methods for encoding and 
            decoding lists of integers. Default is None, which gets replaced
            with UncompressedPostings
        directory (str): Directory where the index files will be stored
        """
        self.index_file_path = os.path.join(directory, index_name+'.index')

        self.metadata_file_path = os.path.join(directory, index_name+'.dict')

        if postings_encoding is None:
            self.postings_encoding = UncompressedPostings
        else:
            self.postings_encoding = postings_encoding
        self.directory = directory

        self.postings_dict = {}
        self.terms = []         #Need to keep track of the order in which the 
                                #terms were inserted. Would be unnecessary 
                                #from Python 3.7 onwards

    def __enter__(self):
        """Opens the index_file and loads metadata upon entering the context"""
        # Open the index file
        self.index_file = open(self.index_file_path, 'rb+')

        # Load the postings dict and terms from the metadata file
        with open(self.metadata_file_path, 'rb') as f:
            self.postings_dict, self.terms = pkl.load(f)
            self.term_iter = self.terms.__iter__()                       

        return self
    
    def __exit__(self, exception_type, exception_value, traceback):
        """Closes the index_file and saves metadata upon exiting the context"""
        # Close the index file
        self.index_file.close()
        
        # Write the postings dict and terms to the metadata file
        with open(self.metadata_file_path, 'wb') as f:
            pkl.dump([self.postings_dict, self.terms], f)

因为是在与磁盘上的文件进行交互，所以我们提供了`__enter__`和`__exit__`函数，它使得我们能够像使用python中文件IO一样使用`with`语句。（参考[上下文管理器官方文档](https://docs.python.org/3/library/contextlib.html)）

以下是使用 `InvertedIndexWriter` 上下文管理器的实例:

```python
with InvertedIndexWriter('test', directory='tmp/') as index:
    # Some code here
```

## 索引

> BSBI (i) segments the collection into parts of equal size, (ii) sorts the termID-docID pairs of each part in memory, (iii) stores intermediate sorted results on disk, and (iv) merges all intermediate results into the final index. 

你需要将每一个子目录当做一个块（block），并且在构建索引的过程中每次只能加载一个块（模拟内存不足）。注意到我们是将操作系统意义上的块进行了抽象。你可以认为每个块足够小，能被装载进内存。

在这一部分，我们将阶段性地构造类`BSBIIndex`。函数`index`给出了BSBI算法的框架，而你的工作则是在接下来的部分中实现函数`parse_block`, `invert_write` 和 `merge`。

In [817]:
# Do not make any changes here, they will be overwritten while grading
class BSBIIndex:
    """ 
    Attributes
    ----------
    term_id_map(IdMap): For mapping terms to termIDs
    doc_id_map(IdMap): For mapping relative paths of documents (eg 
        0/3dradiology.stanford.edu_) to docIDs
    data_dir(str): Path to data
    output_dir(str): Path to output index files
    index_name(str): Name assigned to index
    postings_encoding: Encoding used for storing the postings.
        The default (None) implies UncompressedPostings
    """
    def __init__(self, data_dir, output_dir, index_name = "BSBI", 
                 postings_encoding = None):
        self.term_id_map = IdMap()
        self.doc_id_map = IdMap()
        self.data_dir = data_dir
        self.output_dir = output_dir
        self.index_name = index_name
        self.postings_encoding = postings_encoding

        # Stores names of intermediate indices
        self.intermediate_indices = []
        
    def save(self):
        """Dumps doc_id_map and term_id_map into output directory"""
        
        with open(os.path.join(self.output_dir, 'terms.dict'), 'wb') as f:
            pkl.dump(self.term_id_map, f)
        with open(os.path.join(self.output_dir, 'docs.dict'), 'wb') as f:
            pkl.dump(self.doc_id_map, f)
    
    def load(self):
        """Loads doc_id_map and term_id_map from output directory"""
        
        with open(os.path.join(self.output_dir, 'terms.dict'), 'rb') as f:
            self.term_id_map = pkl.load(f)
        with open(os.path.join(self.output_dir, 'docs.dict'), 'rb') as f:
            self.doc_id_map = pkl.load(f)
            
    def index(self):
        """Base indexing code
        
        This function loops through the data directories, 
        calls parse_block to parse the documents
        calls invert_write, which inverts each block and writes to a new index
        then saves the id maps and calls merge on the intermediate indices
        """
        for block_dir_relative in sorted(next(os.walk(self.data_dir))[1]):
            td_pairs = self.parse_block(block_dir_relative)
            index_id = 'index_'+block_dir_relative
            self.intermediate_indices.append(index_id)
            with InvertedIndexWriter(index_id, directory=self.output_dir, 
                                     postings_encoding=
                                     self.postings_encoding) as index:
                self.invert_write(td_pairs, index)
                td_pairs = None
        self.save()
        with InvertedIndexWriter(self.index_name, directory=self.output_dir, 
                                 postings_encoding=
                                 self.postings_encoding) as merged_index:
            with contextlib.ExitStack() as stack:
                indices = [stack.enter_context(
                    InvertedIndexIterator(index_id, 
                                          directory=self.output_dir, 
                                          postings_encoding=
                                          self.postings_encoding)) 
                 for index_id in self.intermediate_indices]
                self.merge(indices, merged_index)

### 解析

> The function `parse_block`  parses documents into termID-docID pairs and accumulates the pairs in memory until a block of a fixed size is full. We choose the block size to fit comfortably into memory to permit a fast in-memory sort. 

你需要将每一个子目录当做一个块，`parse_block`接收子目录路径作为参数。同一子目录下所有文件名都是不同的。

_注意 - 我们使用 `BSBIIndex` 继承 `BSBIIndex`, 这只是对一个已存在类添加新内容的简单方法。在这里只是用来切分类的定义（jupyter notebook内教学使用，无特殊含义）。_

In [818]:
class BSBIIndex(BSBIIndex):            
    def parse_block(self, block_dir_relative:str):
        """Parses a tokenized text file into termID-docID pairs
        
        Parameters
        ----------
        block_dir_relative : str
            Relative Path to the directory that contains the files for the block
        
        Returns
        -------
        List[Tuple[Int, Int]]
            Returns all the td_pairs extracted from the block
        
        Should use self.term_id_map and self.doc_id_map to get termIDs and docIDs.
        These persist across calls to parse_block
        """
        ### Begin your code
        folder_path = self.data_dir + "/" + block_dir_relative
        tmp_td_pairs = list()
        ###print(folder_path)
        ###print(next(os.walk(folder_path))[2])
        ###读取该文件夹下的所有文件
        for file in next(os.walk(folder_path))[2]:
            with open(folder_path + "/" + file, 'r') as f:
                file_name = os.path.normpath(block_dir_relative + "/" + file)
                ##print(file_name)
                self.doc_id_map[file_name]
                tokens = f.read().split()
                for token in tokens:
                    self.term_id_map[token]
                    tmp_tuple = (self.term_id_map[token],self.doc_id_map[file_name])
                    tmp_td_pairs.append(tmp_tuple) 
            f.close()
            ###print(file_name)
        td_pairs = sorted(list(set(tmp_td_pairs)))
    
        ###print(self.doc_id_map.str_to_id)
        ###print(self.term_id_map.str_to_id)
        ###print(td_pairs)



        ###td_pairs指的是 -> term_id mapped doc_id
        return td_pairs
        ### End your code

观察函数在测试数据上是否正常运行

In [819]:
with open('toy-data/0/fine.txt', 'r') as f:
    print(f.read())
with open('toy-data/0/hello.txt', 'r') as f:
    print(f.read())

i'm fine , thank you

hi hi
how are you ?



In [820]:
BSBI_instance = BSBIIndex(data_dir=toy_dir, output_dir = 'tmp/', index_name = 'toy')
BSBI_instance.parse_block('0')

[(0, 0),
 (1, 0),
 (2, 0),
 (3, 0),
 (4, 0),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1)]

写一些测试样例来确保`parse_block`方法正常运行（如：相同单词出现时是相同ID）

In [821]:
### Begin your code
tmp1 = BSBI_instance.parse_block('0')
tmp2 = BSBI_instance.parse_block('1')
tmp3 = BSBI_instance.parse_block('2')
print(tmp1)
print(tmp2)
print(tmp3)
### End your code

[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]
[(2, 2), (4, 2), (8, 2), (9, 2), (10, 2), (11, 2), (12, 2), (12, 3), (13, 2)]
[(0, 4), (1, 4), (2, 4), (3, 4), (4, 4), (4, 5), (5, 5), (6, 5), (7, 5), (8, 5)]


### 倒排表

> The block is then inverted and written to disk. Inversion involves two steps. First, we sort the termID-docID pairs. Next, we collect all termID-docID pairs with the same termID into a postings list, where a posting is simply a docID. The result, an inverted index for the block we have just read, is then written to disk.

在这一部分我们添加函数`invert_write`来实现由termID-docID对构建倒排表。 

但是，我们首先需要实现类`InvertedIndexWriter`。和列表类似，该类提供了append函数，但是倒排表不会存储在内存中而是直接写入到磁盘里。

In [822]:
class InvertedIndexWriter(InvertedIndex):
    """"""
    def __enter__(self):
        self.index_file = open(self.index_file_path, 'wb+')              
        return self

    def append(self, term, postings_list):
        """Appends the term and postings_list to end of the index file.
        
        This function does three things, 
        1. Encodes the postings_list using self.postings_encoding
        2. Stores metadata in the form of self.terms and self.postings_dict
           Note that self.postings_dict maps termID to a 3 tuple of 
           (start_position_in_index_file, 
           number_of_postings_in_list, 
           length_in_bytes_of_postings_list)
        3. Appends the bytestream to the index file on disk

        Hint: You might find it helpful to read the Python I/O docs
        (https://docs.python.org/3/tutorial/inputoutput.html) for
        information about appending to the end of a file.
        
        Parameters
        ----------
        term:
            term or termID is the unique identifier for the term
        postings_list: List[Int]
            List of docIDs where the term appears
        """
        ### Begin your code
        
        """text files (those opened without a b in the mode string), 
        only seeks relative to the beginning of the file are allowed 
        (the exception being seeking to the very file end with seek(0, 2)) 
        and the only valid offset values are those returned from the f.tell(), or zero.
        Any other offset value produces undefined behaviour.
        从文档中获得的信息，使用.tell()函数得到offset
        """
        ###实现对于postings_list的编码
        encoded_postings_list = self.postings_encoding.encode(postings_list)
        ###获取bytes的长度
        encoded_len = len(encoded_postings_list)

        ###这里其实是当同一个InveredIndexWriter时写入是继续添加的，但不同的是覆盖的
        with open(self.index_file_path, "ab") as index_in:
        ###构建postings_dict
        ###当没有这个term时，添加这个term
            if self.postings_dict.get(term) is None:
                self.terms.append(term)
                self.postings_dict[term] = (index_in.tell(), len(postings_list), encoded_len)
            index_in.write(encoded_postings_list)
            index_in.close()
        ### End your code

尽管还没有实现读取索引的类，我们还是可以用以下测试代码检测我们的实现。

In [823]:
with InvertedIndexWriter('test', directory='tmp/') as index:
    index.append(1, [2, 3, 4])
    index.append(2, [3, 4, 5])
    index.index_file.seek(0)
    assert index.terms == [1,2], "terms sequence incorrect"
    assert index.postings_dict == {1: (0, 3, len(UncompressedPostings.encode([2,3,4]))), 
                                   2: (len(UncompressedPostings.encode([2,3,4])), 3, 
                                       len(UncompressedPostings.encode([3,4,5])))}, "postings_dict incorrect"
    assert UncompressedPostings.decode(index.index_file.read()) == [2, 3, 4, 3, 4, 5], "postings on disk incorrect"

现在我们实现 `invert_write`，它将解析得到的td_pairs转换成倒排表，并使用`InvertedIndexWriter` 类将其写入磁盘。

In [824]:
class BSBIIndex(BSBIIndex):
    def invert_write(self, td_pairs:list[tuple[int,int]], index: InvertedIndexWriter):
        """Inverts td_pairs into postings_lists and writes them to the given index
        
        Parameters
        ----------
        td_pairs: List[Tuple[Int, Int]]
            List of termID-docID pairs
        index: InvertedIndexWriter
            Inverted index on disk corresponding to the block       
        """
        ### Begin your code
        ###由于这里的td_pairs已经排序了，所以只要term和term_id不同，就可以继续
        term_id = td_pairs[0][0]
        postings_list = list()
        with index as index_write:   # 对，就是这
            for tup in td_pairs:
                ###当相等时，添加到postings_list中
                if term_id == tup[0]:
                    postings_list.append(tup[1])
                ###当不相等时，由于已经排序了，所以肯定不再有了
                else:
                    index_write.append(term_id, postings_list)
                    term_id = tup[0]
                    postings_list.clear()
                    postings_list.append(tup[1])
            index_write.append(term_id, postings_list)
        ### End your code

我们可以在测试数据上读取一个块并观察倒排索引中包含的内容。
仿照`InvertedIndexWriter`部分写一些测试样例。

In [825]:
### Begin your code
BSBI_instance = BSBIIndex(data_dir=toy_dir, output_dir = 'tmp/', index_name = 'toy')
td_pairs = BSBI_instance.parse_block('0')
print(td_pairs)
with InvertedIndexWriter(index_name='1', directory=BSBI_instance.output_dir) as index:  
    BSBI_instance.invert_write(td_pairs, index)
with open(os.path.join(BSBI_instance.output_dir, '1.index'), 'rb') as f:
    f.seek(0)
    print(UncompressedPostings.decode(f.read()))
td_pairs = BSBI_instance.parse_block('1')
with InvertedIndexWriter(index_name='1', directory=BSBI_instance.output_dir) as index: 
    BSBI_instance.invert_write(td_pairs, index)
print(td_pairs)
with open(os.path.join(BSBI_instance.output_dir, '1.index'), 'rb') as f:
    f.seek(0)
    print(UncompressedPostings.decode(f.read()))
### End your code

[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]
[0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
[(2, 2), (4, 2), (8, 2), (9, 2), (10, 2), (11, 2), (12, 2), (12, 3), (13, 2)]
[2, 2, 2, 2, 2, 2, 2, 3, 2]


### 合并
> The algorithm simultaneously merges the ten blocks into one large merged index. To do the merging, we open all block files simultaneously, and maintain small read buffers for the ten blocks we are reading and a write buffer for the final merged index we are writing. 

Python中的迭代模型非常自然地符合我们维护一个小的读缓存的要求。我们可以迭代地从磁盘上每次读取文件的一个倒排列表。我们通过构建`InvertedIndex`的子类`InvertedIndexIterator`来完成这个迭代任务。

In [826]:
class InvertedIndexIterator(InvertedIndex):
    """"""
    def __init__(self, index_name, postings_encoding=None, directory=''):
        super().__init__(index_name, postings_encoding, directory)
        self.cnt = 0

    def __enter__(self):
        """Adds an initialization_hook to the __enter__ function of super class
        """
        super().__enter__()
        self._initialization_hook()
        return self

    def _initialization_hook(self):
        """Use this function to initialize the iterator
        """
        ### Begin your code
        ###初始化获取terms的迭代器

        ###哭，迭代器用不了 ?

        ###似乎可以这样写
        ### End your code

    def __iter__(self): 
        return self
    
    def __next__(self):
        """Returns the next (term, postings_list) pair in the index.
        
        Note: This function should only read a small amount of data from the 
        index file. In particular, you should not try to maintain the full 
        index file in memory.
        """
        ### Begin your code
        if self.cnt < len(self.terms):
            term = self.terms[self.cnt]
            with open(self.index_file_path, 'rb') as index:
                ###得到初始位置
                    self.cnt += 1
                    start_position, list_len, byte_len = self.postings_dict[term]
                    index.seek(start_position)
                    encoded_postings_list = index.read(byte_len)
                    postings_list = UncompressedPostings.decode(encoded_postings_list)
            return (term,postings_list)
        else:
            raise StopIteration
        ### End your code

    def delete_from_disk(self):
        """Marks the index for deletion upon exit. Useful for temporary indices
        """
        self.delete_upon_exit = True

    def __exit__(self, exception_type, exception_value, traceback):
        """Delete the index file upon exiting the context along with the
        functions of the super class __exit__ function"""
        self.index_file.close()
        if hasattr(self, 'delete_upon_exit') and self.delete_upon_exit:
            os.remove(self.index_file_path)
            os.remove(self.metadata_file_path)
        else:
            with open(self.metadata_file_path, 'wb') as f:
                pkl.dump([self.postings_dict, self.terms], f)

为了测试以上代码，我们先用`InvertedIndexWriter` 创建索引，然后再迭代遍历。写一些小的测试样例观察它是否正常运行。至少你得写一个测试，手工构建一个小的索引，用`InvertedIndexWriter`将他们写入磁盘，然后用`InvertedIndexIterator`迭代遍历倒排索引。

In [827]:
### Begin your code
with InvertedIndexWriter('test', directory='tmp/') as index:
    index.append(1, [2, 3, 4])
    index.append(2, [3, 4, 5])
    index.index_file.seek(0)
    assert index.terms == [1,2], "terms sequence incorrect"
    assert index.postings_dict == {1: (0, 3, len(UncompressedPostings.encode([2,3,4]))), 
                                   2: (len(UncompressedPostings.encode([2,3,4])), 3, 
                                       len(UncompressedPostings.encode([3,4,5])))}, "postings_dict incorrect"
    assert UncompressedPostings.decode(index.index_file.read()) == [2, 3, 4, 3, 4, 5], "postings on disk incorrect"

with InvertedIndexIterator('test',None,'tmp/') as iter:
    for it in iter:
        print(it)

### End your code

(1, [2, 3, 4])
(2, [3, 4, 5])


> During merging, in each iteration, we select the lowest termID that has not been processed yet using a priority queue or a similar data structure. All postings lists for this termID are read and merged, and the merged list is written back to disk. Each read buffer is refilled from its file when necessary.

我们将使用`InvertedIndexIterator`来完成读取的部分，用`InvertedIndexWriter`来将合并后的索引写入磁盘。

注意到我们之前一直使用`with` 语句来打开倒排索引文件，但是现在需要程序化地完成这项工作。可以看到我们在基类`BSBIIndex`的`index`函数中使用了`contextlib`([参考文档](https://docs.python.org/3/library/contextlib.html#contextlib.ExitStack))
你的任务是合并*打开的*`InvertedIndexIterator`对象（倒排列表），并且通过一个`InvertedIndexWriter`对象每次写入一个文档列表。

既然我们知道文档列表已经排过序了，那么我们可以在线性时间内对它们进行合并排序。事实上`heapq`([参考文档](https://docs.python.org/3.0/library/heapq.html)) 是Python中一个实现了堆排序算法的标准模板。另外它还包含一个实用的函数`heapq.merge`，可以将多个已排好序的输入（可迭代）合并成一个有序的输出并返回它的迭代器。它不仅可以用来合并倒排列表，也可以合并倒排索引词典。

为了让你快速上手`heapq.merge`函数，我们提供了一个简单的使用样例。给出两个以动物和树平均寿命排序的列表，我们希望合并它们。

In [828]:
import heapq
animal_lifespans = [('Giraffe', 28), 
                   ('Rhinoceros', 40), 
                   ('Indian Elephant', 70), 
                   ('Golden Eagle', 80), 
                   ('Box turtle', 123)]

tree_lifespans = [('Gray Birch', 50), 
                  ('Black Willow', 70), 
                  ('Basswood', 100),
                  ('Bald Cypress', 600)]

lifespan_lists = [animal_lifespans, tree_lifespans]

for merged_item in heapq.merge(*lifespan_lists, key=lambda x: x[1]):
    print(merged_item)

('Giraffe', 28)
('Rhinoceros', 40)
('Gray Birch', 50)
('Indian Elephant', 70)
('Black Willow', 70)
('Golden Eagle', 80)
('Basswood', 100)
('Box turtle', 123)
('Bald Cypress', 600)


注意使用`*`将`lifespan_lists`解包作为参数，并且使用`lambda`函数来给出进行排序的key。如果你对它们不熟悉可以参考文档([unpacking lists](https://docs.python.org/3/tutorial/controlflow.html#unpacking-argument-lists), [lambda expressions](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions))。

In [829]:
import heapq
class BSBIIndex(BSBIIndex):
    def merge(self, indices:list[InvertedIndexIterator], merged_index:InvertedIndexWriter):
        """Merges multiple inverted indices into a single index
        
        Parameters
        ----------
        indices: List[InvertedIndexIterator]
            A list of InvertedIndexIterator objects, each representing an
            iterable inverted index for a block
        merged_index: InvertedIndexWriter
            An instance of InvertedIndexWriter object into which each merged 
            postings list is written out one at a time
        """
        ### Begin your code
        ###对于每个Iter都是一个InvertedIndexIterator
        ###再对这个进行遍历，由于是同一个InvertedWriter，故这里可以直接写
        combined_list = list()
        for iter in indices:
            for it in iter:
                term_id = it[0]
                postings_list = it[1]
                tmp_list = [(term_id, postings_list)]
                combined_list.append(tmp_list)
        for merged_item in heapq.merge(*combined_list, key=lambda x: x[0]):
            merged_index.append(merged_item[0], merged_item[1])
        ### End your code

首先确保它在测试数据上正常运行

In [830]:
BSBI_instance = BSBIIndex(data_dir=toy_dir, output_dir = 'toy_output_dir', )
BSBI_instance.index()
with InvertedIndexIterator('BSBI',directory='toy_output_dir') as f:
    for it in f:
        print(it)

(0, [0])
(1, [0])
(2, [0])
(3, [0])
(4, [0, 1])
(5, [1])
(6, [1])
(7, [1])
(8, [1])
(9, [2])
(10, [2])
(11, [2])
(12, [2, 3])
(13, [2])


接下来对整个数据集构建索引

In [831]:
'''BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )
BSBI_instance.index()'''

"BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )\nBSBI_instance.index()"

如果你在合并阶段出现了错误，可以利用以下代码来debug。

In [832]:
"""BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )
BSBI_instance.intermediate_indices = ['index_'+str(i) for i in range(10)]
with InvertedIndexWriter(BSBI_instance.index_name, directory=BSBI_instance.output_dir, postings_encoding=BSBI_instance.postings_encoding) as merged_index:
    with contextlib.ExitStack() as stack:
        indices = [stack.enter_context(InvertedIndexIterator(index_id, directory=BSBI_instance.output_dir, postings_encoding=BSBI_instance.postings_encoding)) for index_id in BSBI_instance.intermediate_indices]
        BSBI_instance.merge(indices, merged_index)"""

"BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )\nBSBI_instance.intermediate_indices = ['index_'+str(i) for i in range(10)]\nwith InvertedIndexWriter(BSBI_instance.index_name, directory=BSBI_instance.output_dir, postings_encoding=BSBI_instance.postings_encoding) as merged_index:\n    with contextlib.ExitStack() as stack:\n        indices = [stack.enter_context(InvertedIndexIterator(index_id, directory=BSBI_instance.output_dir, postings_encoding=BSBI_instance.postings_encoding)) for index_id in BSBI_instance.intermediate_indices]\n        BSBI_instance.merge(indices, merged_index)"

## 布尔联合检索 (10%)

我们将实现BSBIIndex中的`retrieve`函数，对于给定的包含由空格分隔tokens的字符串查询，返回包含查询中所有tokens的文档列表。但是我们并不能迭代遍历整个索引或者将整个索引加载到内存来寻找相应的terms（索引太大）。

首先我们要实现`InvertedIndex`的子类`InvertedIndexMapper`，它能够找到对应terms在索引文件中位置并取出它的倒排记录表。

In [833]:
class InvertedIndexMapper(InvertedIndex):
    def __getitem__(self, key):
        return self._get_postings_list(key)
    
    def _get_postings_list(self, term):
        """Gets a postings list (of docIds) for `term`.
        
        This function should not iterate through the index file.
        I.e., it should only have to read the bytes from the index file
        corresponding to the postings list for the requested term.
        """
        ### Begin your code
        ###当term不在terms中，返回空链表
        if term not in self.terms:
            ###print(self.terms)
            return []
        ###当term在terms中，通过postings_dir找到postings_list
        else:
            ###print(self.postings_dict)
            start_position, num_of_list, encoded_len = self.postings_dict[term]
            with open(self.index_file_path, 'rb') as index:
                index.seek(start_position)
                encoded_postings_list = index.read(encoded_len)
                postings_list = UncompressedPostings.decode(encoded_postings_list)
                return postings_list
        ### End your code

写一些测试样例检测`_get_postings_list`的实现

In [834]:
### Begin your code
with InvertedIndexMapper('BSBI',directory='toy_output_dir') as mapper:
    postings_list = mapper._get_postings_list(0)
    print(postings_list)
    postings_list = mapper._get_postings_list(1)
    print(postings_list)
    postings_list = mapper._get_postings_list(2)
    print(postings_list)
    postings_list = mapper._get_postings_list(3)
    print(postings_list)
    postings_list = mapper._get_postings_list(4)
    print(postings_list)
### End your code

[0]
[0]
[0]
[0]
[0, 1]


现在我们获得了查询中terms对应的倒排记录表，接着需要求他们的交集。这部分与我们之前作业的merge方法类似，请实现`sorted_intersect`函数，遍历两个有序列表并在线性时间内合并。

更正：这里根本不是进行合并（），指导书纯扯淡，这里应进行地两个倒排列表的求交操作，得到共同出现的元素，这样才是我们需要进行查询的文档

In [855]:
def sorted_intersect(list1, list2):
    """Intersects two (ascending) sorted lists and returns the sorted result
    
    Parameters
    ----------
    list1: List[Comparable]
    list2: List[Comparable]
        Sorted lists to be intersected
        
    Returns
    -------
    List[Comparable]
        Sorted intersection        
    """
    ### Begin your code
    cnt1 = 0
    cnt2 = 0
    sorted_list = list()
    ###首先当长度未达到列表长度时，进行内部的添加
    while cnt1 < len(list1) and cnt2 < len(list2):
        if list1[cnt1] == list2[cnt2]:
            sorted_list.append(list1[cnt1])
            cnt1 = cnt1 + 1
            cnt2 = cnt2 + 1
        elif list1[cnt1] > list2[cnt2]:
            sorted_list.append(list2[cnt2])
            cnt2 = cnt2 + 1
        else:
            sorted_list.append(list1[cnt1])
            cnt1 = cnt1 + 1

                    
    ###当这时结束时，将剩下的list添加到后面
    ###更正：这里根本不需要进行操作，直接继续即可
    if cnt1 >= len(list1):
        sorted_list = sorted_list + list2[cnt2:]
    else:
        sorted_list = sorted_list + list1[cnt1:]
    return sorted_list


    ###在这里去了重，但不知道会不会有问题   
    ### End your code

简单的测试样例

In [854]:
### Begin your code
list1 = sorted(['\\5a', 'b', 'saa.,d','2asd'])
list2 = sorted(['23', 'b', '2asd', 'sdada', '\\5a'])
sorted_list = sorted_intersect(list1,list2)
print(sorted_list)
### End your code

IndexError: list index out of range

现在可以利用`sorted_intersect` 和 `InvertedIndexMapper`来实现`retrieve`函数。

In [841]:
###%%tee submission/retrieve.py
class BSBIIndex(BSBIIndex):
    def retrieve(self, query:str):
        """Retrieves the documents corresponding to the conjunctive query
        
        Parameters
        ----------
        query: str
            Space separated list of query tokens
            
        Result
        ------
        List[str]
            Sorted list of documents which contains each of the query tokens. 
            Should be empty if no documents are found.
        
        Should NOT throw errors for terms not in corpus
        """
        if len(self.term_id_map) == 0 or len(self.doc_id_map) == 0:
            self.load()
        

        ### Begin your code
        tokens = query.split()
        my_list = list()
        for token in tokens:
            ###这时得到的是term字符串，需要先转换成term_id
            if self.term_id_map.str_to_id.get(token) is None:
                continue
            term_id = self.term_id_map[token]
            str_list = list()
            ###用于存储doc_str文件名的列表，再遍历求交
            ###根据term_id找到对应的doc_id list
            with InvertedIndexMapper(self.index_name, self.postings_encoding, directory=self.output_dir) as mapper:
                postings_list = mapper._get_postings_list(term_id)
                for postings in postings_list:
                    doc_str = self.doc_id_map[postings]
                    ###这是一个存着string的list
                    str_list.append(doc_str)
                    ###这是一个存着list(str)的list
                my_list.append(str_list)
        ###创建结果链表记录结果
        result_list = list()
        for item in my_list:
            print(item)
        if my_list is None:
            return list()
        else:
            result_list.append(my_list[0][0])
            ###print(result_list)
            for item in my_list:
                ###print(item)
                result_list = sorted_intersect(result_list, item)
            return result_list
        ### End your code

通过一个简单的查询来测试索引在真实数据集上是否正常工作。

In [842]:
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )
BSBI_instance.retrieve('boolean retrieval')

['0\\boole.stanford.edu_pub_uni.pdf', '0\\ccrma.stanford.edu_planetccrma_man_man8_apt-config.8.html', '0\\ccrma.stanford.edu_planetccrma_mirror_centos_linux_planetccrma_5_i386_repoview_supercollider-mathlib.html', '0\\compbio.stanford.edu_research-labs.html', '0\\comptop.stanford.edu_u_programs_jplex_files_doc_edu_stanford_math_plex_SimplexStream.Head.html', '0\\covertlab.stanford.edu_projects_iFBA_']
['0\\acomp.stanford.edu_jobs_technical_support_specialist', '0\\airlab.stanford.edu_workshops_june2011presentations_Asier_June2011OpenHouse.pdf', '0\\arcade.stanford.edu_arcade-journals', '0\\blogs.law.stanford.edu_robotics_2012_03_16_technological-rationality-the-logos-of-slavery-or-the-enabler-of-humane-progress_', '0\\bmir.stanford.edu_pages_view.php_national_center_for_biomedical_ontology_ncbo_receives_funding_through_2015', '0\\bmir.stanford.edu_people_view.php_4368', '0\\bmir.stanford.edu_people_view.php_lawrence_m_fagan', '0\\bmir.stanford.edu_people_view.php_mark_a_musen_', '0\\bm

[]

你也可以通过读取文件来测试其中的页面是否真的包含了查询的terms

In [843]:
with open("pa1-data/1/cs276.stanford.edu_", 'r') as f:
    print(f.read())

cs276 information retrieval and web search cs 276 ling 286 information retrieval and web search spring 2011 pandu nayak and prabhakar raghavan lecture 3 units tu th 4 15 5 30 gates b03 tas sonali aggarwal ivan cui valentin spitkovsky and sandeep sripada staff e mail cs276 spr1011 staff lists stanford edu lectures are also available online and on television through scpd sitn course description basic and advanced techniques for text based information systems efficient text indexing boolean and vector space retrieval models evaluation and interface issues web search including crawling link based algorithms and web metadata text web clustering classification text mining syllabus additional information staff contact information piazzza we strongly recommend students to post questions to the course page on www piazzza com instead of sending emails this forum enables students to discuss the questions they encounter in lectures or assignments here is a quick introduction video email if you hav

测试dev queries（提前构建好的查询与结果）是否正确

In [844]:
for i in range(1, 9):
    with open('dev_queries/query.' + str(i)) as q:
        query = q.read()
        my_results = [os.path.normpath(path) for path in BSBI_instance.retrieve(query)]
        with open('dev_output/' + str(i) + '.out') as o:
            print(my_results)
            reference_results = [os.path.normpath(x.strip()) for x in o.readlines()]
            assert my_results == reference_results, "Results DO NOT match for query: "+query.strip()
        print("Results match for query:", query.strip())

['0\\5-sure.stanford.edu_', '0\\50years.stanford.edu_', '0\\a3cservices.stanford.edu_awards_nominate_', '0\\a3cservices.stanford.edu_lead_', '0\\aa.stanford.edu_', '0\\aa.stanford.edu_about_aviation.php', '0\\aa.stanford.edu_about_control.php', '0\\aa.stanford.edu_admissions_', '0\\aa.stanford.edu_admissions_admittedGrad.php', '0\\aa.stanford.edu_admissions_fellowsAero.php', '0\\aa.stanford.edu_events_50thAnniversary_media_Ying.pdf', '0\\aa.stanford.edu_events_affiliatesMeeting_affiliates2004meeting.php', '0\\aa.stanford.edu_faculty_facultySearch.php', '0\\aa.stanford.edu_giving_index.php', '0\\aaalab.stanford.edu_child_development_dev_dominos.html', '0\\aaalab.stanford.edu_child_development_dev_fractions.html', '0\\aaalab.stanford.edu_child_development_dev_reasoning.html', '0\\aaalab.stanford.edu_face_to_face_interaction_face_agency.html', '0\\aaalab.stanford.edu_imagery_im_complex.html', '0\\aaalab.stanford.edu_imagery_im_tools.html', '0\\aaalab.stanford.edu_teachable_agents_ta_moby.

AssertionError: Results DO NOT match for query: we are

如果出现了错误，可以通过以下代码比较输出与正确结果的差异

In [ ]:
set(my_results) - set(reference_results)

In [846]:
sorted(set(reference_results) - set(my_results))

['0\\50years.stanford.edu_',
 '0\\a3cservices.stanford.edu_lead_',
 '0\\aa.stanford.edu_',
 '0\\aa.stanford.edu_about_control.php',
 '0\\aa.stanford.edu_admissions_',
 '0\\aa.stanford.edu_admissions_admittedGrad.php',
 '0\\aa.stanford.edu_admissions_fellowsAero.php',
 '0\\aa.stanford.edu_events_50thAnniversary_media_Ying.pdf',
 '0\\aa.stanford.edu_faculty_facultySearch.php',
 '0\\aa.stanford.edu_giving_index.php',
 '0\\aaalab.stanford.edu_child_development_dev_dominos.html',
 '0\\aaalab.stanford.edu_child_development_dev_reasoning.html',
 '0\\aaalab.stanford.edu_face_to_face_interaction_face_agency.html',
 '0\\aaalab.stanford.edu_imagery_im_complex.html',
 '0\\aaalab.stanford.edu_teachable_agents_ta_moby.html',
 '0\\aaastop.stanford.edu_patients_',
 '0\\aaastop.stanford.edu_patients_our_goal.html',
 '0\\aaastop.stanford.edu_patients_who_is_eligible.html',
 '0\\aaastop.stanford.edu_physicians_',
 '0\\aacf.stanford.edu_',
 '0\\aagsa.stanford.edu_content_grad-formal-photos-0',
 '0\\aagsa.

确保你构建自己的查询来测试所有可能的边界情况，例如数据集中没有出现的terms，或者拖慢合并的频繁出现的terms。

# 索引压缩  (30%)

在这部分，你将使用可变长字节编码对索引进行压缩。（gap-encoding可选，对gap进行编码）

下面几个Python运算符可能对你有帮助

In [776]:
# Remainder (modulo) operator %

print("10 % 2 = ", 10 % 2)
print("10 % 3 = ", 10 % 3)

# Integer division in Python 3 is done by using two slash signs

print("10 / 3 = ", 10 / 3)
print("10 // 3 = ", 10 // 3)

10 % 2 =  0
10 % 3 =  1
10 / 3 =  3.3333333333333335
10 // 3 =  3


完成下面的`CompressedPostings`类，我们将用它来替换`UncompressedPostings`。关于如何使用gap-encoding和可变长字节编码，你可以参考老师课件或者[Chapter 5](https://nlp.stanford.edu/IR-book/pdf/05comp.pdf)。

In [ ]:
class CompressedPostings:
    #If you need any extra helper methods you can add them here 
    ### Begin your code

    ### End your code
    
    @staticmethod
    def encode(postings_list):
        """Encodes `postings_list` using gap encoding with variable byte 
        encoding for each gap
        
        Parameters
        ----------
        postings_list: List[int]
            The postings list to be encoded
        
        Returns
        -------
        bytes: 
            Bytes reprsentation of the compressed postings list 
            (as produced by `array.tobytes` function)
        """
        ### Begin your code
        
        ### End your code

        
    @staticmethod
    def decode(encoded_postings_list):
        """Decodes a byte representation of compressed postings list
        
        Parameters
        ----------
        encoded_postings_list: bytes
            Bytes representation as produced by `CompressedPostings.encode` 
            
        Returns
        -------
        List[int]
            Decoded postings list (each posting is a docIds)
        """
        ### Begin your code

        ### End your code


首先，如果你实现了额外的辅助函数，写一些测试样例

In [ ]:
### Begin your code

### End your code

我们实现了一个简单的函数来测试你编码的列表是否被正确解码

In [ ]:
def test_encode_decode(l):
    e = CompressedPostings.encode(l)
    d = CompressedPostings.decode(e)
    assert d == l
    print(l, e)

写一些测试样例来确保文档列表的压缩和解压正常运行

In [ ]:
### Begin your code

### End your code

现在让我们创建一个新的输出文件夹并使用`CompressedPostings`来构建压缩索引

In [ ]:
try: 
    os.mkdir('output_dir_compressed')
except FileExistsError:
    pass

In [ ]:
BSBI_instance_compressed = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir_compressed', postings_encoding=CompressedPostings)
BSBI_instance_compressed.index()

In [ ]:
BSBI_instance_compressed = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir_compressed', postings_encoding=CompressedPostings)
BSBI_instance_compressed.retrieve('boolean retrieval')

像之前一样，用已构造好的查询语句来测试是否正常运行

In [ ]:
for i in range(1, 9):
    with open('dev_queries/query.' + str(i)) as q:
        query = q.read()
        my_results = [os.path.normpath(path) for path in BSBI_instance_compressed.retrieve(query)]
        with open('dev_output/' + str(i) + '.out') as o:
            reference_results = [os.path.normpath(x.strip()) for x in o.readlines()]
            assert my_results == reference_results, "Results DO NOT match for query: "+query.strip()
        print("Results match for query:", query.strip())

# 额外的编码方式 (10%)

通过补充`ECCompressedPostings`的`encode` 和 `decode`方法来实现一种额外的索引压缩方式。在我们课上学的就是**gamma-encoding** 。另外如果大家感兴趣的话也可以了解**Delta Encoding** ，[ALGORITHM SIMPLE-9](https://github.com/manning/CompressionAlgorithms#simple-9) 等。

你应该以多字节（而不是bits）来存储倒排记录表，因为索引的长度和位置都存的是字节信息。

In [ ]:
class ECCompressedPostings:
    #If you need any extra helper methods you can add them here 
    ### Begin your code

    ### End your code
    
    @staticmethod
    def encode(postings_list):
        """Encodes `postings_list` 
        
        Parameters
        ----------
        postings_list: List[int]
            The postings list to be encoded
        
        Returns
        -------
        bytes: 
            Bytes reprsentation of the compressed postings list 
        """
        ### Begin your code

        ### End your code

        
    @staticmethod
    def decode(encoded_postings_list):
        """Decodes a byte representation of compressed postings list
        
        Parameters
        ----------
        encoded_postings_list: bytes
            Bytes representation as produced by `CompressedPostings.encode` 
            
        Returns
        -------
        List[int]
            Decoded postings list (each posting is a docId)
        """
        ### Begin your code

        ### End your code

# 作业提交

本次作业用时y约3周，截止日期为9.30（国庆节前）。请大家在截止日期前将代码（包含运行结果，测试内容不作要求），实验报告（可单独撰写，也可整合在jupyter notebook中）一起提交到ir24fall@163.com，邮件和文件命名方式均为`学号_姓名_hw1`，如`1811412_戚晓睿_hw1.ipynb`